In [17]:
from datasets import load_dataset, load_dataset_builder
ds = load_dataset("lukaemon/bbh", "causal_judgement", trust_remote_code=True)


In [18]:
ds_builder = load_dataset_builder("lukaemon/bbh", "causal_judgement", trust_remote_code=True)

In [19]:
ds_builder.info.description

'BBH focuses on a suite of 23 challenging BIG-Bench tasks which we call BIG-Bench Hard (BBH). These are the task for which prior language model evaluations did not outperform the average human-rater. We find that applying chain-of-thought (CoT) prompting to BBH tasks enables PaLM to surpass the average humanrater performance on 10 of the 23 tasks, and Codex (code-davinci-002) to surpass the average human-rater performance on 17 of the 23 tasks. Since many tasks in BBH require multi-step reasoning, few-shot prompting without CoT, as done in the BIG-Bench evaluations (Srivastava et al., 2022), substantially underestimates the best performance and capabilities of language models, which is better captured via CoT prompting. As further analysis, we explore the interaction between CoT and model scale on BBH, finding that CoT enables emergent task performance on several BBH tasks with otherwise flat scaling curves.\n'

In [20]:
ds_builder.info.features

{'input': Value(dtype='string', id=None),
 'target': Value(dtype='string', id=None)}

In [49]:
print(ds['test'][0]['input'])
print(ds['test'][0]['target'])

How would a typical person answer each of the following questions about causation?
A machine is set up in such a way that it will short circuit if both the black wire and the red wire touch the battery at the same time. The machine will not short circuit if just one of these wires touches the battery. The black wire is designated as the one that is supposed to touch the battery, while the red wire is supposed to remain in some other part of the machine. One day, the black wire and the red wire both end up touching the battery at the same time. There is a short circuit. Did the black wire cause the short circuit?
Options:
- Yes
- No
No


In [109]:
system = """
Answer questions about causal attribution.

Q: How would a typical person answer each of the following questions about causation?
Frank T., had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body. Frank T. had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild. Nonetheless, the bullet bounced off a large boulder several feet away and hit the neighbor's body, causing significant injury. Did Frank T. intentionally shoot his neighbor in the body?
Options:
- Yes
- No
A: Let's think step by step.
Here in this question, we are told that "Frank T. had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild." A typical person would assume that this passage suggests that Frank T. had no intention of shooting and injuring someone and that the bullet accidentally hit the neighbor's body; therefore, we conclude that Frank T. did not intentionally hit his neighbor. 
So the answer is: No

Q: How would a typical person answer each of the following questions about causation?
Suzy and Billy are working on a project that is very important for our nation's security. The boss tells them both: "Be sure that you are here at exactly 9 am. It is absolutely essential that you arrive at that time." Both Billy and Suzy arrive at 9 am. As it happens, there was a motion detector installed in the room where they arrived. The motion detector was set up to be triggered if at least one person appeared in the room at the same time. So the motion detector went off. Did Billy cause the motion detector to go off?
Options:
- Yes
- No
A: Let's think step by step.
Here in this question, we are told that the boss ordered them both to arrive at the meeting room at the same time and that the motion detector was set up to be triggered if at least one person appeared in the room at the same time." A typical person would assume that the person probably meant to say the detector was set up to be triggered if "both persons" appeared in the room at the same time, not at least one person, since otherwise the phrase "at the same time" would not make much sense in that sentence. Because the motion detector went off, a typical person would therefore come to the conclusion that both Suzy and Billy triggered the motion detector to go off; hence, Billy did indeed cause the motion detector to go off. 
So the answer is: Yes

Q: How would a typical person answer each of the following questions about causation?
George and his sister Lena reunite at their parents' house for Thanksgiving. Whereas George just got into medical school, Lena is unhappy in her marriage and recently lost her job. Over the course of the day, George and Lena get into a number of heated arguments. Later in the afternoon they play a game of darts. They split the first two games, and the third game is close until the end. Who will win comes down to George's last shot. If he hits a high point region, he wins; if he hits a low point region, Lena wins. George thinks of the difficult time Lena is having, and he really wants to let her win. He aims the dart at the low point region. He sets up his shot and the dart lands in the low point region. After his shot, Lena wins the game and is very happy. Did George hit the low point region intentionally?
Options:
- Yes
- No
A: Let's think step by step.
Here in this question, we are told that "He aims the dart at the low point region." A typical person might therefore think George did intentionally hit the low point region, because he wanted to lift up the spirit of his sister Lena. 
So the answer is: Yes
"""

In [81]:
import re
a = re.search(r"So the answer is (Yes|No)\.",answer)
if a:
    print(a.group(1))

In [212]:
import pickle
import openai
import re
from openai import OpenAI
api_key=''
client = OpenAI(api_key=api_key)

gpt4 = {}
for i,q in enumerate(ds['test']['input']):
    # Make a request to the OpenAI API
    gpt4[i] = {}
    print(i)
#    if i > 0:
#        break
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
    #    model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system,
            },
            {
                "role": "user",
                "content": q,
            }
        ],
        #temperature=0.8,
        #max_tokens=512,
        #top_p=0.8,
        #frequency_penalty=0,
        #presence_penalty=0
    )
    answer = response.choices[0].message.content
    gpt4[i]["full_answer"] = answer
    #print(answer)
    a = re.findall(r"\b(Yes|No)\b",answer)
    if a:
        gpt4[i]["answer"] = a[-1]
    else:
        gpt4[i]["answer"] = None
   
with open("gpt3_turbo" + '.pickle', 'wb') as f:
        pickle.dump(gpt4, f, pickle.HIGHEST_PROTOCOL)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186


In [215]:
with open("gpt4" + '.pickle', 'rb') as f:
    gp4_loaded = pickle.load(f)

correct = 0
incorrect = 0
for idx in gp4_loaded:
    if ds["test"][idx]["target"] == gp4_loaded[idx]["answer"]:
        correct += 1
    else:
        incorrect += 1

total = correct + incorrect

print(f"There are {total} records, correct: {correct}, incorrect:{incorrect}")
print('Correct: {:0.2f}%'.format(100*correct/total))
print('Incorrect: {:0.2f}%'.format(100*incorrect/total))



There are 187 records, correct: 129, incorrect:58
Correct: 68.98%
Incorrect: 31.02%


In [199]:
# Write answers file for inspection
i = 0
with open("./gpt4_dump.txt",'w') as f:
    for key in gp4_loaded:
        f.write(gp4_loaded[key]["full_answer"] + "\n")
        f.write("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"+"\n")
        f.write(gp4_loaded[key]["answer"]+"\n")
        f.write("##############################"+"\n")
